In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0
%reload_ext autoreload
!pip install -U imbalanced-learn
import threadpoolctl
threadpoolctl.threadpool_info()
%pip install optuna

In [0]:
%pip install xgboost
%pip install lightgbm


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import preprocessing
from preprocessing import *
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, matthews_corrcoef, precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [0]:
data= pd.read_csv("emi_prediction_dataset.csv", low_memory=False)
pre=quality_check(data, Prediction=False)
n_data = pre.final_processsing().emi_pred

In [0]:
target_cols=['emi_eligibility', 'max_monthly_emi']

In [0]:
n_data["emi_eligibility"].value_counts()

In [0]:
n_data["Savings"] = n_data["bank_balance"] + n_data["emergency_fund"]

In [0]:
n_data.head()

In [0]:
def data_prepare(data, datatype="type1", target_type=None):
    final_columns_v1 = [
    'gender', 'marital_status', 'education', 'monthly_salary',
       'employment_type', 'years_of_employment', 'company_type', 'house_type',
       'monthly_rent', 'family_size', 'dependents', 'school_fees',
       'college_fees', 'travel_expenses', 'groceries_utilities',
       'other_monthly_expenses', 'existing_loans', 'current_emi_amount',
       'credit_score', 'bank_balance', 'emergency_fund', 'emi_scenario',
       'requested_amount', 'requested_tenure']
    final_columns_v2=[
     'gender', 'marital_status', 'education', 'monthly_salary',
       'employment_type', 'years_of_employment', 'company_type', 'house_type',
       'monthly_rent', 'family_size', 'dependents',  'existing_loans', 'credit_score', 'emi_scenario',
       'requested_amount', 'requested_tenure', 'Total_Fixed_expenses',
       'Total_Other_expenses','Savings']
    if datatype=="type1":
        if target_type==None:
            final_data=data[final_columns_v1]
        elif target_type=="regression":
            final_data=data[final_columns_v1]
            final_data['max_monthly_emi']=data["max_monthly_emi"]
        elif target_type=="classification":
            final_data=data[final_columns_v1]
            final_data['emi_eligibility']=data["emi_eligibility"]
        else:
            print("Please enter correct target type")
    elif datatype=="type2":
        if target_type==None:
            final_data=data[final_columns_v2]
        elif target_type=="regression":
            final_data=data[final_columns_v2]
            final_data['max_monthly_emi']=data["max_monthly_emi"]
        elif target_type=="classification":
            final_data=data[final_columns_v2]
            final_data['emi_eligibility']=data["emi_eligibility"]
    return final_data

In [0]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
def data_split(x_data,y_data, rebalance = False):
    X=x_data
    y=y_data
    if rebalance==True:
        sm = SMOTE(random_state=42)
        X, y = sm.fit_resample(X, y)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    return x_train, x_test, y_train, y_test

In [0]:
import pickle
with open("assets/v1_scaler.pkl", "rb") as f:
    v1_scaler = pickle.load(f)
with open("assets/v2_scaler.pkl", "rb") as f:
    v2_scaler = pickle.load(f)

In [0]:
data_type1_c=data_prepare(n_data, target_type="classification", datatype="type1")
data_type2_c=data_prepare(n_data, target_type="classification", datatype="type2")


In [0]:
data_type1_r=data_prepare(n_data, target_type="regression", datatype="type1")
data_type2_r=data_prepare(n_data, target_type="regression", datatype="type2")


In [0]:
x_dtype_1=data_type1_c.drop("emi_eligibility", axis=1)
x_dtype_2=data_type2_c.drop("emi_eligibility", axis=1)
y_c=data_type2_c["emi_eligibility"]
y_r=data_type2_r["max_monthly_emi"]

In [0]:
import mlflow
mlflow.set_registry_uri("databricks-uc")
import optuna

In [0]:
import mlflow
import numpy as np
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score,
    explained_variance_score,
    median_absolute_error
)

def log_regression_metrics(y_true, y_pred):
    """Logs standard regression metrics to MLflow."""
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    r2 = r2_score(y_true, y_pred)
    evs = explained_variance_score(y_true, y_pred)
    medae = median_absolute_error(y_true, y_pred)

    mlflow.log_metrics({
        "rmse": rmse,
        "mae": mae,
        "mape(%)": mape,
        "r2": r2,
        "explained_variance": evs,
        "median_ae": medae
    })


In [0]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, matthews_corrcoef, precision_recall_fscore_support
def logging(y_test, pred, model_name="model"):
    # --- 1️⃣ Core Metrics ---
    mlflow.log_metric("accuracy", accuracy_score(y_test, pred))
    mlflow.log_metric("f1_score_weighted", f1_score(y_test, pred, average='weighted'))
    mlflow.log_metric("recall_weighted", recall_score(y_test, pred, average='weighted'))
    mlflow.log_metric("precision_weighted", precision_score(y_test, pred, average='weighted'))

    balanced_acc = balanced_accuracy_score(y_test, pred)
    kappa = cohen_kappa_score(y_test, pred)
    mcc = matthews_corrcoef(y_test, pred)

    mlflow.log_metric("balanced_accuracy", balanced_acc)
    mlflow.log_metric("cohen_kappa", kappa)
    mlflow.log_metric("mcc", mcc)

    # --- 2️⃣ Confusion Matrix ---
    cm = confusion_matrix(y_test, pred, labels=np.unique(y_test))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_test))

    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(cmap='Blues', ax=ax, colorbar=False)
    plt.title(f"Confusion Matrix - {model_name}")

    mlflow.log_figure(fig, f"{model_name}_confusion_matrix.png")
    plt.close(fig)

    # --- 3️⃣ Per-Class Metrics ---
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_test, pred, average=None, labels=np.unique(y_test)
    )

    for i, label in enumerate(np.unique(y_test)):
        mlflow.log_metric(f"precision_class_{label}", prec[i])
        mlflow.log_metric(f"recall_class_{label}", rec[i])
        mlflow.log_metric(f"f1_class_{label}", f1[i])

    # --- 4️⃣ Log CM as artifact ---
    #mlflow.log_artifact(figure=f"{model_name}_confusion_matrix.png")

    print(f"✅ Logged metrics and confusion matrix for {model_name}")

In [0]:
experiment_name= "/Workspace/Users/sushantkashikar1@gmail.com/mlflow/Final_model_Training"
try:
    mlflow.create_experiment(experiment_name)
except:
    print("Experiment_already Exists")
mlflow.set_experiment(experiment_name)


In [0]:
from lightgbm import LGBMClassifier

In [0]:
from mlflow.models import infer_signature

In [0]:
from lightgbm import LGBMRegressor
# or from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_dtype_1, y_r, test_size=0.2, random_state=42)
run_name="lightgbm_regressor"
with mlflow.start_run(run_name=run_name):
    best_params = {
    "n_estimators": 1000,
    "learning_rate": 0.03,
    "num_leaves": 50,
    "max_depth": 8,
    "min_child_samples": 20,
    "subsample": 0.8,                # same as bagging_fraction
    "colsample_bytree": 0.8,         # same as feature_fraction
    "reg_alpha": 0.1,                # L1 regularization
    "reg_lambda": 0.2,               # L2 regularization
    "random_state": 42,
    "n_jobs": -1}
    model = LGBMRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    log_regression_metrics(y_test, y_pred)
    mlflow.sklearn.log_model(model, "LGBMRegressor",signature= infer_signature(X_test, y_test))


In [0]:
from lightgbm import LGBMRegressor
# or from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_dtype_2, y_r, test_size=0.2, random_state=42)
run_name="lightgbm_regressor_dtype2"
with mlflow.start_run(run_name=run_name):
    best_params = {
    "n_estimators": 1000,
    "learning_rate": 0.03,
    "num_leaves": 50,
    "max_depth": 8,
    "min_child_samples": 20,
    "subsample": 0.8,                # same as bagging_fraction
    "colsample_bytree": 0.8,         # same as feature_fraction
    "reg_alpha": 0.1,                # L1 regularization
    "reg_lambda": 0.2,               # L2 regularization
    "random_state": 42,
    "n_jobs": -1}
    model = LGBMRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    log_regression_metrics(y_test, y_pred)
    mlflow.sklearn.log_model(model, "LGBMRegressor",signature= infer_signature(X_test, y_test))


In [0]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_dtype_1, y_r, test_size=0.2, random_state=42)
run_name = "xgboost_regressor_dtype1"
with mlflow.start_run(run_name=run_name):
    best_params = {
        "n_estimators": 1000,
        "learning_rate": 0.03,
        "max_depth": 8,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "reg_alpha": 0.1,
        "reg_lambda": 0.2,
        "random_state": 42,
        "n_jobs": -1
    }
    model = XGBRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    log_regression_metrics(y_test, y_pred)
    mlflow.sklearn.log_model(model, "XGBRegressor", signature= infer_signature(X_test, y_test))

In [0]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_dtype_2, y_r, test_size=0.2, random_state=42)
run_name = "xgboost_regressor_dtype2"
with mlflow.start_run(run_name=run_name):
    best_params = {
        "n_estimators": 1000,
        "learning_rate": 0.03,
        "max_depth": 8,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "reg_alpha": 0.1,
        "reg_lambda": 0.2,
        "random_state": 42,
        "n_jobs": -1
    }
    model = XGBRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mlflow.log_params(best_params)
    log_regression_metrics(y_test, y_pred)
    mlflow.sklearn.log_model(model, "XGBRegressor", signature= infer_signature(X_test, y_test))

In [0]:
x_dtype_2.head(2)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x_dtype_2, y_r, test_size=0.2, random_state=42)
best_params = {
        "n_estimators": 1000,
        "learning_rate": 0.03,
        "max_depth": 8,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "reg_alpha": 0.1,
        "reg_lambda": 0.2,
        "random_state": 42,
        "n_jobs": -1
    }
regressor_model = XGBRegressor(**best_params)
regressor_model.fit(X_train, y_train)
y_pred = regressor_model.predict(X_test)

In [0]:
import pickle
pickle.dump(regressor_model, open("save_models/regressor_model.pkl", "wb"))

Classification ML

In [0]:
from lightgbm import LGBMClassifier
# or from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_dtype_1, y_c, test_size=0.2, random_state=42)
run_name="lightgbm_Classification_dtype1"
with mlflow.start_run(run_name=run_name):
    lgbm_best_params = {
    "n_estimators": 800,
    "learning_rate": 0.05,
    "num_leaves": 45,
    "max_depth": 8,
    "min_child_samples": 20,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 0.1,
    "reg_lambda": 0.5,
    "is_unbalance": True,
    "random_state": 42,
    "n_jobs": -1}

    model = LGBMClassifier(**lgbm_best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    logging(y_test, y_pred)
    mlflow.sklearn.log_model(model, "LGBMclassifier",signature= infer_signature(X_test, y_test))


In [0]:
from lightgbm import LGBMClassifier
# or from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_dtype_2, y_c, test_size=0.2, random_state=42)
run_name="lightgbm_Classification_dtype2"
with mlflow.start_run(run_name=run_name):
    lgbm_best_params = {
    "n_estimators": 800,
    "learning_rate": 0.05,
    "num_leaves": 45,
    "max_depth": 8,
    "min_child_samples": 20,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 0.1,
    "reg_lambda": 0.5,
    "is_unbalance": True,
    "random_state": 42,
    "n_jobs": -1}

    model = LGBMClassifier(**lgbm_best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    logging(y_test, y_pred)
    mlflow.sklearn.log_model(model, "LGBMclassifier",signature= infer_signature(X_test, y_test))


In [0]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = data_split(x_dtype_1, y_c, rebalance=True)
from sklearn.utils.class_weight import compute_class_weight
run_name = "xgboost_classifier_dtype1"
with mlflow.start_run(run_name=run_name):
    classes = np.unique(y_train)
    class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=classes, 
    y=y_train)
    class_weights_dict = dict(zip(classes, class_weights))
    print("📊 Class Weights:", class_weights_dict)

# 2️⃣ Define model parameters
    xgb_multiclass_params = {
        "n_estimators": 800,
        "learning_rate": 0.05,
        "max_depth": 7,
        "min_child_weight": 3,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "gamma": 0.1,
        "reg_alpha": 0.1,
        "reg_lambda": 1.0,
        "use_label_encoder": False,
        "objective": "multi:softprob",    # 👈 multiclass objective
        "num_class": len(classes),        # 👈 number of classes
        "eval_metric": "mlogloss",        # 👈 better metric for multiclass
        "random_state": 42,
        "n_jobs": -1}

    # 3️⃣ Initialize model
    model = XGBClassifier(**xgb_multiclass_params)

    # 4️⃣ Pass sample weights to handle imbalance
    sample_weights = np.array([class_weights_dict[label] for label in y_train])


    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    logging(y_test, y_pred)
    mlflow.sklearn.log_model(model, "XGBclassifier", signature= infer_signature(X_test, y_test))

In [0]:
target_cols

In [0]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
new_x_dtype1=x_dtype_1.copy()
new_x_dtype1["max_monthly_emi"]=y_r
X_train, X_test, y_train, y_test = data_split(new_x_dtype1, y_c, rebalance=True)
from sklearn.utils.class_weight import compute_class_weight
run_name = "xgboost_classifier_dtype1"
with mlflow.start_run(run_name=run_name):
    classes = np.unique(y_train)
    class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=classes, 
    y=y_train)
    class_weights_dict = dict(zip(classes, class_weights))
    print("📊 Class Weights:", class_weights_dict)

# 2️⃣ Define model parameters
    xgb_multiclass_params = {
        "n_estimators": 800,
        "learning_rate": 0.05,
        "max_depth": 7,
        "min_child_weight": 3,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "gamma": 0.1,
        "reg_alpha": 0.1,
        "reg_lambda": 1.0,
        "use_label_encoder": False,
        "objective": "multi:softprob",    # 👈 multiclass objective
        "num_class": len(classes),        # 👈 number of classes
        "eval_metric": "mlogloss",        # 👈 better metric for multiclass
        "random_state": 42,
        "n_jobs": -1}

    # 3️⃣ Initialize model
    model = XGBClassifier(**xgb_multiclass_params)

    # 4️⃣ Pass sample weights to handle imbalance
    sample_weights = np.array([class_weights_dict[label] for label in y_train])


    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    logging(y_test, y_pred)
    mlflow.sklearn.log_model(model, "XGBclassifier", signature= infer_signature(X_test, y_test))

In [0]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
new_x_dtype2=x_dtype_2.copy()
new_x_dtype2["max_monthly_emi"]=y_r
X_train, X_test, y_train, y_test = data_split(new_x_dtype2, y_c, rebalance=True)
from sklearn.utils.class_weight import compute_class_weight
run_name = "xgboost_classifier_dtype2"
with mlflow.start_run(run_name=run_name):
    classes = np.unique(y_train)
    class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=classes, 
    y=y_train)
    class_weights_dict = dict(zip(classes, class_weights))
    print("📊 Class Weights:", class_weights_dict)

# 2️⃣ Define model parameters
    xgb_multiclass_params = {
        "n_estimators": 800,
        "learning_rate": 0.05,
        "max_depth": 7,
        "min_child_weight": 3,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "gamma": 0.1,
        "reg_alpha": 0.1,
        "reg_lambda": 1.0,
        "use_label_encoder": False,
        "objective": "multi:softprob",    # 👈 multiclass objective
        "num_class": len(classes),        # 👈 number of classes
        "eval_metric": "mlogloss",        # 👈 better metric for multiclass
        "random_state": 42,
        "n_jobs": -1}

    # 3️⃣ Initialize model
    model = XGBClassifier(**xgb_multiclass_params)

    # 4️⃣ Pass sample weights to handle imbalance
    sample_weights = np.array([class_weights_dict[label] for label in y_train])


    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    logging(y_test, y_pred)
    mlflow.sklearn.log_model(model, "XGBclassifier", signature= infer_signature(X_test, y_test))

In [0]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
new_x_dtype1=x_dtype_1.copy()
new_x_dtype1["max_monthly_emi"]=y_r

X_train, X_test, y_train, y_test = train_test_split(new_x_dtype1, y_c, test_size=0.2, random_state=42)
from sklearn.utils.class_weight import compute_class_weight
run_name = "xgboost_classifier_dtype1"
with mlflow.start_run(run_name=run_name):
    classes = np.unique(y_train)
    class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=classes, 
    y=y_train)
    class_weights_dict = dict(zip(classes, class_weights))
    print("📊 Class Weights:", class_weights_dict)

# 2️⃣ Define model parameters
    xgb_multiclass_params = {
        "n_estimators": 800,
        "learning_rate": 0.05,
        "max_depth": 7,
        "min_child_weight": 3,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "gamma": 0.1,
        "reg_alpha": 0.1,
        "reg_lambda": 1.0,
        "use_label_encoder": False,
        "objective": "multi:softprob",    # 👈 multiclass objective
        "num_class": len(classes),        # 👈 number of classes
        "eval_metric": "mlogloss",        # 👈 better metric for multiclass
        "random_state": 42,
        "n_jobs": -1}

    # 3️⃣ Initialize model
    model = XGBClassifier(**xgb_multiclass_params)

    # 4️⃣ Pass sample weights to handle imbalance
    sample_weights = np.array([class_weights_dict[label] for label in y_train])


    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    logging(y_test, y_pred)
    mlflow.sklearn.log_model(model, "XGBclassifier", signature= infer_signature(X_test, y_test))

In [0]:
from lightgbm import LGBMClassifier
# or from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
new_x_dtype2=x_dtype_2.copy()
new_x_dtype2["max_monthly_emi"]=y_r

X_train, X_test, y_train, y_test = data_split(new_x_dtype2, y_c, rebalance=True)
run_name="lightgbm_Classification_dtype2"
with mlflow.start_run(run_name=run_name):
    lgbm_best_params = {
    "n_estimators": 800,
    "learning_rate": 0.05,
    "num_leaves": 45,
    "max_depth": 8,
    "min_child_samples": 20,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 0.1,
    "reg_lambda": 0.5,
    "is_unbalance": True,
    "random_state": 42,
    "n_jobs": -1}
    mlflow.log_params(lgbm_best_params)
    model = LGBMClassifier(**lgbm_best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    logging(y_test, y_pred)
    mlflow.sklearn.log_model(model, "LGBMclassifier",signature= infer_signature(X_test, y_test))


In [0]:
n_data

In [0]:
new_x_dtype2=x_dtype_2.copy()
new_x_dtype2["max_monthly_emi"]=y_r

X_train, X_test, y_train, y_test = data_split(new_x_dtype2, y_c, rebalance=True)

lgbm_best_params = {
    "n_estimators": 800,
    "learning_rate": 0.05,
    "num_leaves": 45,
    "max_depth": 8,
    "min_child_samples": 20,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 0.1,
    "reg_lambda": 0.5,
    "is_unbalance": True,
    "random_state": 42,
    "n_jobs": -1}
classifier_model = LGBMClassifier(**lgbm_best_params)
classifier_model.fit(X_train, y_train)
y_pred = classifier_model.predict(X_test)

In [0]:
import pickle
pickle.dump(classifier_model, open('save_models/classifier_model.pkl','wb'))